In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import re
import numpy as np
import pandas as pd
import nltk
import nltk
from nltk.corpus import stopwords
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [3]:
# Getting path of folder stories and folders inside it

getPath = str('/content/drive/MyDrive/stories')
print(getPath)
getAllFolders = [x[0] for x in os.walk(getPath)]

getAllFolders.pop(len(getAllFolders)-1)
print(getAllFolders)

/content/drive/MyDrive/stories
['/content/drive/MyDrive/stories', '/content/drive/MyDrive/stories/SRE', '/content/drive/MyDrive/stories/FARNON']


In [4]:
#After analysing we found index.html that contains all the file names and titles
# create a tuple of all files path and title

list_of_all_files = []
flag = 0
for i in getAllFolders:
    path = i+"/index.html"
    print(path)
    openIndexFile = open(path, 'r')
    getContent = openIndexFile.read().strip()
    openIndexFile.close()
    
    # file name is enclosed in Anchor tag of html file
    getFileName = re.findall('><A HREF="(.*)">', getContent)
    #print(getFileName)
    
    # file title is enclosed in <BR><TD> tag of html file
    getFileTitle = re.findall('<BR><TD> (.*)\n', getContent)
    #print(len(getFileTitle))
    
    #for excluding folder inside stories which are at 0th and 1st index
    if flag==0:
        getFileName = getFileName[2:]
        flag = 1
    for j in range(len(getFileName)):
        list_of_all_files.append((str(i)+"/" + str(getFileName[j]), getFileTitle[j]))
    
#print(len(list_of_all_files))

/content/drive/MyDrive/stories/index.html
/content/drive/MyDrive/stories/SRE/index.html
/content/drive/MyDrive/stories/FARNON/index.html


In [5]:
#Convert data in to lower case
def lower(text):
    return np.char.lower(text)

# remove all the stopwords(‘off’, ‘is’, ‘s’, ‘am’, ‘or’, ‘who’, e.t.c) from data
def stopWords(text):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(text))
    new_text = ""
    for w in words:
        if w not in stop_words:
            new_text = new_text + " " + w
    return np.char.strip(new_text)

# remove all the punctuation symbols
def punctuationSymbols(text):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        text = np.char.replace(text, symbols[i], ' ')
        text = np.char.replace(text, "  ", " ")
        text = np.char.replace(text, ',', '')
    return text

# remove all apostrophes  
def unUsed(text):
  return np.char.replace(text,"'","")

# def deleteNumerics(text):
#     text = np.char.replace(text, "0", " zero ")
#     text = np.char.replace(text, "1", " one ")
#     text = np.char.replace(text, "2", " two ")
#     text = np.char.replace(text, "3", " three ")
#     text = np.char.replace(text, "4", " four ")
#     text = np.char.replace(text, "5", " five ")
#     text = np.char.replace(text, "6", " six ")
#     text = np.char.replace(text, "7", " seven ")
#     text = np.char.replace(text, "8", " eight ")
#     text = np.char.replace(text, "9", " nine ")
#     return text


In [6]:
#call all the above functions to preprocess the data
def preprocess(data,query):    
    data = lower(data)
    data = stopWords(data)
    data = punctuationSymbols(data) #remove comma seperately
    # data = deleteNumerics(data)
    data = unUsed(data)
    return data





In [7]:
num = 0
content = {}
doc_map = {}
pos_index = {}
for i in list_of_all_files:
    file = open(i[0], 'r', encoding='cp1250', errors='ignore')
    text = file.read().strip()
    content[num] = i[1]
    doc_map[num] = i[1]
    file.close()
    preprocessed_text = preprocess(text, False)

    if num%100==0:
      print(num)

    preprocessed_text=str(preprocessed_text)
    tokens = preprocessed_text.split()
    count = 0
    for token in tokens:
      if token not in pos_index:
        pos_index[token] = []
        pos_index[token].append(1)
        pos_index[token].append({})
        pos_index[token][1][num] = []
        pos_index[token][1][num].append(count)
        count+=1
      else:
        #print(pos_index[token][1])
        if num in pos_index[token][1]:
          pos_index[token][1][num].append(count)
          pos_index[token][0] = len(pos_index[token][1]) 
        else:
          pos_index[token][0]+=1
          pos_index[token].append({})
          pos_index[token][1][num] = []
          pos_index[token][1][num].append(count)

        count+=1

      # print(pos_index)
    num+=1


0
100
200
300
400


In [8]:
print(pos_index['good'])
print(pos_index['days'])

[296, {0: [666], 1: [30, 40, 85, 133, 239, 306], 6: [837], 7: [817, 912], 8: [324, 732, 734, 743, 1013, 1701, 1747, 1967, 2136, 2336, 2463, 2559, 2861], 9: [525], 11: [539, 901, 1222, 1614, 1824, 1831, 1883, 1900, 2032, 2843], 13: [84, 114, 351, 717], 14: [963, 2517], 16: [1420, 1455, 1548, 1550, 1990, 2573, 3431, 3557, 3927, 3988], 17: [203], 18: [73, 507, 1480, 1859, 2231], 19: [560, 848], 20: [1438, 1575, 1860, 2084, 3313, 3488, 3652, 3687, 3951], 22: [1089, 1145], 23: [624], 26: [44, 139, 1342, 1450, 1913, 2121, 2574, 3090, 3892, 4464, 4790, 4847, 4869, 5822, 6120, 6586, 6965, 7229, 7924, 8279, 8287, 8644, 10274, 10587, 10594, 10603, 11123, 11685, 11702, 11716, 11735, 12160, 12286, 12386, 12692, 12790, 14116, 14751, 14880, 15573], 27: [762, 800, 1555, 2111, 2215, 3099, 3281, 3359, 4066, 4068, 4088, 4096, 4460, 4484, 4770, 5725], 31: [715, 941, 1071, 1175, 1352], 32: [67], 33: [169, 541, 566, 868, 1578, 1892, 1895, 1957, 2018, 2114], 35: [161], 36: [127, 139, 1004, 1027, 1052], 39: 

In [9]:
# print(test_phrase_lemmat)

# print(test_query)

In [10]:
# print(words['good'])
# print(words['day'])

In [13]:
query = input("Enter Query : ")
preprocessed_query = preprocess(query, False)
print(preprocessed_query)
test_query = word_tokenize(str(preprocessed_query))



final = {}
count=0
for word in test_query :
    
    dic_doc = pos_index[word][1]
    
    if len(final) == 0 :
        final = dic_doc
    else:
        listOfDocs = list(set(final.keys()) & set(dic_doc.keys()))
        temp = {}
        
        for i in listOfDocs :
            temp[i] = final[i]    
        final = {}    
        final = temp
        if len(listOfDocs) == 0:
            print("No documents")
            break
        
        for i in listOfDocs :
            
            list_posting = final[i]
            listOfTempPostings = []
            
            for index in list_posting:
                
                if index+count in dic_doc[i]:
                    listOfTempPostings.append(index)
            
            final[i] = listOfTempPostings
            
            if len(final[i]) == 0 :
                del final[i]
        
    count = count + 1

Enter Query : good day
good day


In [16]:
# Length of documents retrived
print("Number of Documents retrived are :" ,len(final))
for k in final:
  print("Document number : ",k,"Document name : ",doc_map[k])

Number of Documents retrived are : 21
Document number :  1 Document name :  The Story of the Sly Fox
Document number :  26 Document name :  Aesop's Fables Translated by George Fyler Townsend
Document number :  27 Document name :  Aesop's Fables (84 of Them) from The PaperLess Readers Club
Document number :  73 Document name :  Brain Damage, by Howard I Cannon, 1979
Document number :  77 Document name :  Breaks: The Adventures of Richard Nixon in the 21st Century, by Philip H. Farber (1992-1993)
Document number :  79 Document name :  The Adventure of the Bruce-Partington Plans
Document number :  153 Document name :  The Enchanted Duplicator, by Walt Willis and Bob Shaw
Document number :  162 Document name :  The Blab Fantasy Novel
Document number :  163 Document name :  The Blab Fantasy novel-Ripped off and edited by The Slipped Disk
Document number :  174 Document name :  The Imp, by Ed Davis (1993)
Document number :  183 Document name :  Forgotten Souls by CAC/The Stinker
Document num